In [1]:
!pip -q install langchain langchain_community langchain-groq tiktoken chromadb huggingface_hub transformers sentence-transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip show langchain

Name: langchain
Version: 0.1.20
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Texts
- Embedding
- ChromaDB
- QA Chain
- Source info

## Setting up LangChain


In [3]:
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain_groq import ChatGroq
from glob import glob
import os


## Models to use

In [4]:
embedding_name = 'nomic-ai/nomic-embed-text-v1.5'

In [5]:
# Load embedding
model_kwargs = {'trust_remote_code': True}
embedding = HuggingFaceEmbeddings(model_name=embedding_name, model_kwargs=model_kwargs)

modules.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/69.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

## Load multiple and process documents

In [6]:
# Initialize documents and path
documents = []
root  = os.getcwd()

# Sort the file names
paths = sorted(glob('*.txt'))

# Copy each file to new file
for path in paths:
  loader = TextLoader(path)
  document = loader.load()
  documents.extend(document)

In [7]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1800, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [8]:
len(texts)

285

In [9]:
for text in texts:
  print(text.metadata)

{'source': 'A00-Abbreviations.txt'}
{'source': 'A00-Abbreviations.txt'}
{'source': 'A00-Abbreviations.txt'}
{'source': 'A01-Description.txt'}
{'source': 'A01-Description.txt'}
{'source': 'A01-Description.txt'}
{'source': 'A02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': 'A02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': 'A02-Catholic Identity, Vision Mission and Core Values.txt'}
{'source': 'A03-The University Seal.txt'}
{'source': 'A04-History.txt'}
{'source': 'A04-History.txt'}
{'source': 'A04-History.txt'}
{'source': 'A04-History.txt'}
{'source': 'A04-History.txt'}
{'source': 'A04-History.txt'}
{'source': 'A05-University Saints.txt'}
{'source': 'A05-University Saints.txt'}
{'source': 'A05-University Saints.txt'}
{'source': 'A05-University Saints.txt'}
{'source': 'A05-University Saints.txt'}
{'source': 'A06-USC Organizational Structure.txt'}
{'source': 'A06-USC Organizational Structure.txt'}
{'source': 'A06-USC Organizational Structure.txt'}

## Create the DB

In [10]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db_v2'

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [11]:
# persiste the db to disk
vectordb.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(
